In [1]:
import fitz
import re
import json

In [2]:
footer_pattern = r"\(\d+\) (DO [LC] \d+ de \d{1,2}\.\d{1,2}\.\d{4}, p\. \d+\.)"
article_split_pattern = r"^(?=Artículo \d+$)"
wrong_new_lines_pattern = r"(?<=[a-z,])\n(?=[a-z](?!\)))"
parentheses_enum_fix = r"(?<=[\w\d])\n(?=\))"
parentheses_split_pattern = r"^(?=[\d]+\)$\n)"

In [3]:
def clean_source_text(source_text: str):
    source_text = re.sub(wrong_new_lines_pattern, " ", source_text)
    source_text = re.sub(footer_pattern, "\n", source_text)
    source_text = re.sub(wrong_new_lines_pattern, " ", source_text)
    source_text = re.sub(parentheses_enum_fix, "", source_text)
    return source_text


def split_by_article(source_text: str) -> dict:
    # First block is preamble
    articles = re.split(article_split_pattern, source_text, flags=re.MULTILINE)[1:]
    articles = [article.strip() for article in articles]
    articles_dict = {f"articulo_{i+1}": art for i, art in enumerate(articles)}
    return articles_dict


def split_by_parentheses_enum_list(source_text: str):
    # First block is preamble
    elems = re.split(parentheses_split_pattern, source_text, flags=re.MULTILINE)[1:]
    elems = [elem.strip() for elem in elems]
    elems_dict = {i: art for i, art in enumerate(elems)}
    return elems_dict

# Original document

In [5]:
with fitz.open('../data/directiva_residuos/2008_98_ce_boetxt.pdf') as doc:
    out_text_2008= ""
    for page in doc:
        text = page.get_text()
        out_text_2008 = out_text_2008 + "\n\n" + text

In [5]:
fix_n_text_2008 = re.sub(wrong_new_lines_pattern, ' ', out_text_2008)

In [6]:
# First block is preamble
articles_2008 = re.split(article_split_pattern, fix_n_text_2008, flags=re.MULTILINE)
articles_2008 = [article.strip() for article in articles_2008]

In [7]:
len(articles_2008)

44

In [8]:
print(articles_2008[5])

Artículo 5
Subproductos
1.
Una sustancia u objeto, resultante de un proceso de producción, cuya finalidad primaria no sea la producción de esa sustancia u objeto, puede ser considerada como subproducto y no como residuo con arreglo al artículo
3, punto
1,
únicamente si se cumplen las siguientes condiciones:
a)
es seguro que la sustancia u objeto va a ser utilizado ulteriormente;
b) la sustancia u objeto puede utilizarse directamente sin tener que someterse a una transformación ulterior distinta de la práctica industrial normal;


c) la sustancia u objeto se produce como parte integrante de un proceso de producción; y
d) el uso ulterior es legal, es decir la sustancia u objeto cumple todos los requisitos pertinentes para la aplicación específica relativos a los productos y a la protección del medio ambiente y de la salud, y no producirá impactos generales adversos para el medio ambiente o la salud humana.
2.
Basándose en las condiciones establecidas en el apartado 1, podrán adoptarse me

In [9]:
print(articles_2008[5])

Artículo 5
Subproductos
1.
Una sustancia u objeto, resultante de un proceso de producción, cuya finalidad primaria no sea la producción de esa sustancia u objeto, puede ser considerada como subproducto y no como residuo con arreglo al artículo
3, punto
1,
únicamente si se cumplen las siguientes condiciones:
a)
es seguro que la sustancia u objeto va a ser utilizado ulteriormente;
b) la sustancia u objeto puede utilizarse directamente sin tener que someterse a una transformación ulterior distinta de la práctica industrial normal;


c) la sustancia u objeto se produce como parte integrante de un proceso de producción; y
d) el uso ulterior es legal, es decir la sustancia u objeto cumple todos los requisitos pertinentes para la aplicación específica relativos a los productos y a la protección del medio ambiente y de la salud, y no producirá impactos generales adversos para el medio ambiente o la salud humana.
2.
Basándose en las condiciones establecidas en el apartado 1, podrán adoptarse me

In [10]:
articles_2008_dict = {f"articulo_{i+1}":art for i, art in enumerate(articles_2008[1:])}

In [12]:
with open('../data/directiva_residuos/articles_2008.json', "w") as json_file:
    json.dump(articles_2008_dict, json_file)

In [6]:
# dense calls
out_text_2008_clean = clean_source_text(out_text_2008)
articles_2008 = split_by_article(out_text_2008_clean)

# Modifications document

In [10]:
with fitz.open('../data/directiva_residuos/2018_851_boetxt.pdf') as doc:
    out_text_2018 = ""
    for page in doc:
        text = page.get_text()
        out_text_2018 = out_text_2018 + "\n\n" + text

In [206]:
fix_n_text_2018 = re.sub(wrong_new_lines_pattern, ' ', out_text_2018)
fix_n_text_2018 = re.sub(parentheses_enum_fix, '', fix_n_text_2018)

In [207]:
# First block is preamble
articles_2018 = re.split(article_split_pattern, fix_n_text_2018, flags=re.MULTILINE)
articles_2018 = [article.strip() for article in articles_2018]

In [208]:
len(articles_2018)

2

In [210]:
# First block is preamble
mods_2018 = re.split(parentheses_split_pattern, articles_2018[1], flags=re.MULTILINE)
mods_2018 = [modifications.strip() for modifications in mods_2018]

In [211]:
len(mods_2018)

33

In [216]:
print(mods_2018[31])

31)
En el anexo II, las operaciones R 3, R 4 y R 5 se sustituyen por el texto siguiente:
«R 3
Reciclado o regeneración de sustancias orgánicas que no se utilizan como disolventes (incluidos el compostaje y otros procesos de transformación biológica) (*11)
R 4
Reciclado o regeneración de metales y de compuestos metálicos
(*12)


R 5
Reciclado o regeneración de otras materias inorgánicas (*13)
(*11)
Incluye la preparación para la reutilización, la gasificación y la pirólisis que utilizan los componentes como elementos químicos, y la valorización de materiales orgánicos en forma de relleno."
(*12) Incluye la preparación para la reutilización."
(*13)
Incluye la preparación para la reutilización, el reciclado de materiales de construcción inorgánicos, la valorización de materiales orgánicos en forma de relleno y la limpieza del suelo que tenga como resultado la valorización del suelo.»."


In [224]:
mods_2018_dict = {i:art for i, art in enumerate(mods_2018[1:])}

In [225]:
with open('../data/directiva_residuos/mods_2018.json', "w") as json_file:
    json.dump(mods_2018_dict, json_file)

In [16]:
# dense calls
out_text_2018_clean = clean_source_text(out_text_2018)
articles_2018 = split_by_article(out_text_2018_clean)
mods_2018 = split_by_parentheses_enum_list(articles_2018["articulo_1"])

In [17]:
mods_2018

{0: '1)\nEl artículo 1 se sustituye por el texto siguiente:\n«Artículo 1\nObjeto y ámbito de aplicación\nLa presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de la generación de residuos y de los impactos negativos de la generación y gestión de los residuos, mediante la reducción del impacto global del uso de los recursos y mediante la mejora de la eficiencia de dicho uso, elementos cruciales para efectuar la transición a una economía circular y garantizar la competitividad de la Unión a largo plazo.».',
 1: '2)\nEn el artículo 2, apartado 2, se añade la letra siguiente:\n«e)\nsustancias que se destinen a ser utilizadas como materias primas para piensos tal como se definen en el artículo 3, apartado 2, letra g), del\nReglamento (CE) n.o 767/2009 del Parlamento Europeo y del Consejo (*1) y que no sean subproductos animales ni los contengan;\n(*1) Reglamento (CE) n.o 767/2009 del Parlamento Europeo y del\n